In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e4_path = kagglehub.competition_download('playground-series-s5e4')

print('Data source import complete.')


100%|██████████| 22.3M/22.3M [00:00<00:00, 131MB/s]

Extracting files...


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
    # This assumes your playground_series_s5e4_path is set in a prior cell
print(playground_series_s5e4_path)
print(os.listdir(playground_series_s5e4_path))

/root/.cache/kagglehub/competitions/playground-series-s5e4
['test.csv', 'sample_submission.csv', 'train.csv']


In [ ]:
!pip install xgboost
!pip install lightgbm

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 36.5 MB/s eta 0:00:00


In [ ]:
train = pd.read_csv("/root/.cache/kagglehub/competitions/playground-series-s5e4/train.csv")
test = pd.read_csv("/root/.cache/kagglehub/competitions/playground-series-s5e4/test.csv")
sample_submission = pd.read_csv("/root/.cache/kagglehub/competitions/playground-series-s5e4/sample_submission.csv")

In [ ]:
target = train["Listening_Time_minutes"]
train.drop(columns=["Listening_Time_minutes"], inplace=True)

train["is_train"] = 1
test["is_train"] = 0

data = pd.concat([train, test], axis=0)
data.drop(columns=["Podcast_Name", "Episode_Title"], inplace=True)

# Label encode categorical columns
for col in data.select_dtypes(include='object').columns:
    data[col] = LabelEncoder().fit_transform(data[col])

# Fill missing values
data.fillna(data.median(numeric_only=True), inplace=True)

In [ ]:
train_data = data[data["is_train"] == 1].drop(columns=["is_train", "id"])
test_data = data[data["is_train"] == 0].drop(columns=["is_train", "id"])

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, target, test_size=0.2, random_state=42)

In [ ]:
# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_valid)

# XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_valid)

# LightGBM
lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)
lgb_preds = lgb_model.predict(X_valid)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 794
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 8
[LightGBM] [Info] Start training from score 45.447808


In [ ]:
ensemble_preds = (rf_preds + xgb_preds + lgb_preds) / 3

mse = mean_squared_error(y_valid, ensemble_preds)  # Calculate MSE
rmse = np.sqrt(mse)  # Calculate RMSE

print("Ensemble RMSE:", rmse)

Ensemble RMSE: 12.797418357467306


In [ ]:
rf_test = rf.predict(test_data)
xgb_test = xgb_model.predict(test_data)
lgb_test = lgb_model.predict(test_data)

final_preds = (rf_test + xgb_test + lgb_test) / 3

sample_submission["Listening_Time_minutes"] = final_preds
sample_submission.to_csv("ensemble_submission.csv", index=False)